In [2]:
import numpy as np
import pandas as pd
import csv
import cv2
import numpy as np
import random
import os
from tqdm import tqdm

import torch
import torch.nn as nn
import torchvision.models
import torchvision.transforms as trans
from torch.utils.data import Dataset, DataLoader

In [3]:
'''
path define block
'''
TRAIN_PATH = "train"
TEST_PATH = "test"
#device = "cuda"
device = torch.device("cuda", 1)
data_multiple = 5
# try device = "cuda" 
# and change your settings/accelerator to GPU if you want it to run faster

In [4]:
'''
self-define dataset 
'''
class multi_Dataset(Dataset):
    def __init__(self, data, root, return_filename=False, len = 1):
        self.data = data
        self.return_filename = return_filename
        self.root = root
        self.len = len
    
    def __getitem__(self, index):
        filename, label, ID = self.data[index]

        label_code = []
        for character in label:
            if(character.isalpha()):label_code.append(ord(character)- ord('a') + 10)
            else: label_code.append(int(character))

        img = cv2.imread(f"{self.root}/{filename}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (64,64))
        #img = np.mean(img, axis=2) # take the means of color
        process = trans.Compose([
                            trans.ToTensor(), 
	                        trans.Normalize(mean = [0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                            ])
        img = process(img)
        if self.return_filename:
            img = trans.functional.rotate(img, (random.randint(0,data_multiple-1) - data_multiple/2)*10, fill = 255)
            return img, filename
        else:
            img = trans.functional.rotate(img, (ID-data_multiple/2)*10, fill = 255)
            #plt.imshow(  img.permute(1, 2, 0)  )
            return img, label_code

    def __len__(self):
        return len(self.data)

In [5]:
'''
task1 model
'''
class task1_Resnet50(torchvision.models.resnet.ResNet):
    def __init__(self):
        # Pass default resnet50 arguments to super init
        # https://github.com/pytorch/vision/blob/e130c6cca88160b6bf7fea9b8bc251601a1a75c5/torchvision/models/resnet.py#L260
        super(task1_Resnet50, self).__init__(torchvision.models.resnet.Bottleneck, [3, 4, 6, 3])
        self.load_state_dict(torchvision.models.resnet50(pretrained=True).state_dict())
        self.mylayer1 = nn.Linear(2048, 512)
        self.mylayer2 = nn.Linear(512, 10)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.mylayer1(x)
        x = self.mylayer2(x)

        return [x]

    def forward(self, x):
        return self._forward_impl(x)

In [6]:
'''
task2 model
'''
class task2_Resnet50(torchvision.models.resnet.ResNet):
    def __init__(self, pretrained=True, len = 2):
        # Pass default resnet50 arguments to super init
        # https://github.com/pytorch/vision/blob/e130c6cca88160b6bf7fea9b8bc251601a1a75c5/torchvision/models/resnet.py#L260
        super(task2_Resnet50, self).__init__(torchvision.models.resnet.Bottleneck, [3, 4, 6, 3])
        self.load_state_dict(torchvision.models.resnet50(pretrained=True).state_dict())
        self.mylayer1 = nn.Linear(2048, 512)
        self.len = len
        self.mylayer2A = nn.Linear(512, 36)
        self.mylayer2B = nn.Linear(512, 36)
        
    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.mylayer1(x)
        x1 = self.mylayer2A(x)
        x2 = self.mylayer2B(x)

        return [x1, x2]

    def forward(self, x):
        return self._forward_impl(x)

In [7]:
'''
task3 model
'''
class task3_Resnet50(torchvision.models.resnet.ResNet):
    def __init__(self, pretrained=True, len = 2):
        # Pass default resnet50 arguments to super init
        # https://github.com/pytorch/vision/blob/e130c6cca88160b6bf7fea9b8bc251601a1a75c5/torchvision/models/resnet.py#L260
        super(task3_Resnet50, self).__init__(torchvision.models.resnet.Bottleneck, [3, 4, 6, 3])
        self.load_state_dict(torchvision.models.resnet50(pretrained=True).state_dict())
        self.mylayer1 = nn.Linear(2048, 512)
        self.len = len
        self.mylayer2A = nn.Linear(512, 36)
        self.mylayer2B = nn.Linear(512, 36)
        self.mylayer2C = nn.Linear(512, 36)
        self.mylayer2D = nn.Linear(512, 36)
        '''
        self.mylayerA = nn.Linear(512, 36)
        self.mylayerB = nn.Linear(512, 36)
        '''
    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.mylayer1(x)
        x1 = self.mylayer2A(x)
        x2 = self.mylayer2B(x)
        x3 = self.mylayer2C(x)
        x4 = self.mylayer2D(x)

        return [x1, x2, x3, x4]

    def forward(self, x):
        return self._forward_impl(x)

In [8]:
'''
model inference block
'''

test_model = [0]*3
test_model[0] = task1_Resnet50().to(device)
test_model[1] = task2_Resnet50(len = 2).to(device) 
test_model[2] = task3_Resnet50(len = 4).to(device) 

test_model[0].load_state_dict(torch.load(f'109550110/task1_model_1.pt'))
test_model[1].load_state_dict(torch.load(f'109550110/task2_model_1.pt'))
test_model[2].load_state_dict(torch.load(f'109550110/task3_model_1.pt'))

test_model[0].eval()
test_model[1].eval()
test_model[2].eval()

test_data = []
with open(f'{TEST_PATH}/../sample_submission.csv', newline='') as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
      if(row[0] == "filename"): continue
      test_data.append(row + [-1])  

test_ds_1 = multi_Dataset([data for data in test_data if data[0].startswith("task1")], root=TEST_PATH, return_filename=True,  len = 1)
test_dl_1 = DataLoader(test_ds_1, batch_size=500, num_workers=1, drop_last=False, shuffle=False)

test_ds_2 = multi_Dataset([data for data in test_data if data[0].startswith("task2")], root=TEST_PATH, return_filename=True, len = 2)
test_dl_2 = DataLoader(test_ds_2, batch_size=500, num_workers=1, drop_last=False, shuffle=False)

test_ds_3 = multi_Dataset([data for data in test_data if data[0].startswith("task3")], root=TEST_PATH, return_filename=True,  len = 4)
test_dl_3 = DataLoader(test_ds_3, batch_size=500, num_workers=1, drop_last=False, shuffle=False)

if os.path.exists('submission.csv'):
    csv_writer = csv.writer(open('109550110_submission.csv', 'w', newline=''))
else:
    csv_writer = csv.writer(open('109550110_submission.csv', 'w', newline=''))
csv_writer.writerow(["filename", "label"])

for j, dl in enumerate([test_dl_1, test_dl_2, test_dl_3]):
    print(j)
    for image, filenames in dl:
        image = image.to(device)
        pred = test_model[j](image)
        for i, p in enumerate(pred):
            pred[i] = torch.argmax(p, dim=1)

        #print(len(filenames))
        for i in range(len(filenames)):
            answer = ''
            for p in pred:
                if(p[i].item() >= 10): answer += str(chr(p[i].item() - 10 + ord('a')))
                else: answer += str(p[i].item())
            csv_writer.writerow([filenames[i], answer])

d:\colding\virtualenv\ML_HW5\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\colding\virtualenv\ML_HW5\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\R5 3600/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

AssertionError: Torch not compiled with CUDA enabled